# Tarea 3: Derivación automática 1

Fecha de envío del PR inicial: **lunes 27 de marzo, antes de la clase**

Fecha de aceptación del PR: **martes 4 de abril, antes de la clase**

## Ejercicio 1

Implementen una nueva estructura paramétrica (`type`) que llamaremos `Dual` y que defina los duales, donde el parámetro debe ser un subtipo de `Real`; la siguiente celda sirve para empezar. La parte que identifica a $f_0$ será llamada `fun`, y la correspondiente a $f'_0$ será `der`.

La definición debe incluir métodos que sean compatibles con las dos propiedades arriba mencionadas, es decir, que el dual de una constante (cualquier número real) sea $(c,0)$, y que el de la variable independiente sea $(x_0,1)$. Para lo segundo definiremos una función `xdual` con la propiedad mencionada.

---

In [1]:
"""
    Dual{T<:Real}

Definición de los duales, donde lo campos son:
...
"""
type Dual{T<:Real}
    fun :: T
    der :: T
end

Dual

In [2]:
#= 
Definan un método que permita la promoción automática cuando 
las entradas para definir el dual no son del mismo tipo
=#
function Dual(f, d)
    return Dual(promote(f, d)...)
end

Dual{T<:Real}

In [3]:
#= 
Aquí se define un método que garantiza que el dual de una constante 
(número) cumple lo requerido
=#

function Dual(a::Real)
    return Dual(a, 0)
end

Dual{T<:Real}

In [4]:
#= 
Aquí se define la función `xdual`, que se usará para identificar 
la variable independiente. La función dependerá de x_0, y debe 
regresar el Dual apropiado a la variable independiente
=#

"""
    xdual(x0) -> Dual(x0, 1)

...
"""
function xdual(x0)
        return Dual(x0,1)
end

xdual

In [5]:
methods(Dual)

# 4 methods for generic function "(::Type)":
(::Type{Dual})(a::Real) at In[3]:7
(::Type{Dual}){T<:Real}(fun::T, der::T) at In[1]:8
(::Type{Dual})(f, d) at In[2]:6
(::Type{T}){T}(arg) at sysimg.jl:53

In [6]:
#= 
Muestren que su código funciona con tests adecuados para crear duales,
para promoverlos, y al definir el dual de un número y `xdual`.

En esto es útil usar la infraestructura de Julia; ver:
https://julia.readthedocs.io/en/stable/stdlib/test/

using Base.Test

a = Dual(1, 2.0)
@test a.fun == 1.0
@test a.der == 2.0

=#
using Base.Test

a = Dual(1, 2.0) # Tes de funcion promote
@test a.fun == 1.0
@test a.der == 2.0

b = Dual(3)

@test b.fun == 3
@test b.der == 0


x0 = 10

c = xdual(x0)

@test c.fun == 10
@test c.der == 1

Test Passed
  Expression: c.der == 1
   Evaluated: 1 == 1

# Aritmética de duales

De la definición, y recordando el significado de cada una de las componentes del par doble, tenemos que las operaciones aritméticas quedan definidas por:

\begin{eqnarray}
    \vec{u} \pm \vec{w} &=& \big( u_0 \pm w_0, \, u'_0\pm w'_0 \big),\\
    \vec{u} \times \vec{w} &=& \big( u_0 \cdot w_0,\, u_0 w'_0 +  w_0 u'_0 \big),\\
    \frac{\vec{u}}{\vec{w}} &=& \big( \frac{u_0}{w_0},\, \frac{ u'_0 - (u_0/w_0)w'_0}{w_0}\big),\\
    {\vec{u}}^\alpha &=& \big( u_0^\alpha,\, \alpha u_0^{\alpha-1} u'_0 \big).\\
\end{eqnarray}    

Noten que las dos últimas operaciones son compatibles con $\hat{\mathbf{h}}^2=0$.

Además, están los operadores unitarios, que satisfacen:
\begin{equation}
    \pm \vec{u} = \big(\pm u_0, \pm u'_0 \big).
\end{equation}    

## Ejercicio 2

Implementen *todas* las operaciones aritméticas definidas arriba. Estas operaciones deben incluir las operaciones aritméticas que involucran un número cualquiera (`a :: Real`) y un dual (`b::Dual`), como por ejemplo `a+b` o `b+a`, etc. Esto se puede hacer implementando los métodos específicos para estos casos (¡y que sirven en cualquier órden!). 

Implementen también la comparación entre duales (`==`). 

Incluyan tests que muestren que cada una de ellas está bien definida, y que sus resultados dan valores consistentes.

In [7]:
-(3)

-3

In [29]:
import Base: +, -, *, /, ^, ==

#= 
Aquí se implementan los métodos necesarios para cada función; 
en el caso de ^ por ahora nos conformaremos con que la potencia 
sea entera.
=#

+(a::Dual, b::Dual) = Dual(a.fun + b.fun, a.der + b.der)
+(a::Dual, b::Real) = Dual(a.fun + b, a.der) #se agregan las operaciones respecto a reales.
+(b::Real, a::Dual) = Dual(a.fun + b, a.der)

-(a::Dual, b::Dual) = Dual(a.fun - b.fun, a.der - b.der)  
-(a::Dual, b::Real) = Dual(a.fun - b, a.der)
-(b::Real, a::Dual) = Dual(a.fun - b, a.der)

*(a::Dual, b::Dual) = Dual(a.fun * b.fun, a.fun * b.der + b.fun * a.der) # Aplicando regla del producto d(f*g) = df * g + dg * f
*(a::Dual, b::Real) = Dual(a.fun * b, a.der * b)
*(a::Real, b::Dual) = Dual(a * b.fun, a * b.der)

/(a::Dual, b::Dual) = Dual(a.fun / b.fun, (a.der - (a.fun / b.fun) * b.der) / b.fun)
/(a::Dual, b::Real) = Dual(a.fun / b, a.der / b)
/(a::Real, b::Dual) = Dual(a / b.fun, -a * b.der / b.fun^2)

^(a::Dual, b::Int) = Dual(a.fun^b, b * a.der * a.fun^(b - 1))

+(a::Dual) = Dual(+a.fun, +a.der) # Operador unitario +, +(Dual) = Dual(+, +)
-(a::Dual) = Dual(-a.fun, -a.der) # Operador unitario +, -(Dual) = Dual(-, -)

==(a::Dual, b::Dual) = (a.fun == b.fun) && (a.der == b.der)

== (generic function with 119 methods)

In [56]:
# Aquí se incluyen las pruebas necesarias

a = Dual(8, 4) # Utilizo estos 3 duales porque creo que cubren todos los casos de uso.
b = Dual(3)    # (3, 0)
c = xdual(27)  # (27, 1)
d = 4          # Real.

s1 = a + b     # (8, 4) + (3,  0) = (11, 4)
s2 = a + c     # (8, 4) + (27, 1) = (35, 5)
s3 = b + c     # (3, 0) + (27, 1) = (30, 1)
s4 = a + d     # (8, 4) + 4       = (12, 4)
s5 = + a       #        + (8,  4) = (+8, +4)

r1 = a - b     # (8, 4) - (3,  0) = (5, 4)
r2 = a - c     # (8, 4) - (27, 1) = (-19, 3)
r3 = b - c     # (3, 0) - (27, 1) = (-24, -1)
r4 = a - d     # (8, 4) - 4       = (4, 4)
r5 = - a       #        - (8, 4)  = (-8, -4)

p1 = a * b     # (8, 4) * (3,  0) = (24, 12)
p2 = a * c     # (8, 4) * (27, 1) = (216, 116)
p3 = b * c     # (3, 0) * (27, 1) = (81, 3)
p4 = a * d     # (8, 4) * 4       = (32, 16)

d1 = a / b     # (8, 4) / (3,  0) = (8 / 3, 4 / 3)
d2 = a / c     # (8, 4) / (27, 1) = (8 / 27, (4 * 27 - 8) / 27^2)
d3 = b / c     # (3, 0) / (27, 1) = (3 / 27, -3 / 27^2)
d4 = a / d     # (8, 4) / 4       = (2, 1)

pot1 = a^d     # (8, 4) ^ 4 = (4096, 256)

Dual{Int64}(4096,8192)

In [57]:
# Test de suma

@test s1.fun == 11
@test s1.der == 4

@test s2.fun == 35
@test s2.der == 5

@test s3.fun == 30
@test s3.der == 1

@test s4.fun == 12
@test s4.der == 4

@test s5.fun == 8
@test s5.der == 4

# Test de resta

@test r1.fun == 5
@test r1.der == 4

@test r2.fun == -19
@test r2.der == 3

@test r3.fun == -24
@test r3.der == -1

@test r4.fun == 4
@test r4.der == 4

@test r5.fun == -8
@test r5.der == -4

# Test de producto

@test p1.fun == 24
@test p1.der == 12

@test p2.fun == 216
@test p2.der == 116

@test p3.fun == 81
@test p3.der == 3

@test p4.fun == 32
@test p4.der == 16

# Test de division

@test d1.fun == 2.6666666666666665
@test d1.der == 1.3333333333333333

@test d2.fun == 0.2962962962962963
@test d2.der == 0.13717421124828533

@test d3.fun == 0.1111111111111111
@test d3.der == -0.004115226337448559

@test d4.fun == 2
@test d4.der == 1

# Test de potencia

@test pot1.fun == 4096
@test pot1.der == 8192


Test Passed
  Expression: pot1.der == 8192
   Evaluated: 8192 == 8192

## Ejercicio 3

Para reutilizar el código que han hecho en este notebook, y de hecho seguirlo desarrollando, conviene ponerlo dentro de un módulo. Para hacer esto, deberán copiar todo el código necesario (y que aparece en la resolución de los ejercicios anteriores) en un archivo cuyo nombre será "AutomDiff.jl" y cuya estructura será la siguiente

```julia
#=
Aquí viene una explicación de lo que se hace en el módulo, 
los autores y la fecha
=#

# La siguiente instrucción sirve para *precompilar* el módulo
__precompile__(true)

module AD
    import Base: +, -, *, /, ^, ==
    
    export Dual, xdual
    
    # Aquí viene TODO el código que implementaron.
    # Primero uno incluye la definición de Dual y
    # después las operaciones con Duales.
    ...

end
```

Todas las pruebas deberán ser incluidas en un archivo separado llamado "runtest.jl", y su estructura es:

```julia
# Este archivo incluye los tests del módulo AD
include("AutomDiff.jl")
using Base.test
using AD

# A continuación vienen los tests que implementaron y que deben 
# ser suficientemente exhaustivos
...

```

Estos dos archivos deben incluirlos en el envío de su tarea (además del archivo `Tarea3.ipynb`).

In [58]:
?Base

search: Base base basename base64encode base64decode Base64EncodePipe



No documentation found.

No `README.md` found for module `Base`.
